**Traducteur anglais vers français**

In [ ]:
import os, sys
from tensorflow import keras

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import bsr_matrix

In [ ]:
BATCH_SIZE = 64                     # taille des batchs pour le train
EPOCHS = 20                         # Nombre d'époque pour le tarin  
LSTM_NODES =256                     # taille des units pour les LSTM
NUM_SENTENCES = 100 #20000          # nombre de phrases pour construire le traducteur
MAX_SENTENCE_LENGTH = 50            # nombre de mots max sur une phrase
MAX_NUM_WORDS = 20000               # taille du vocabulaire
EMBEDDING_SIZE = 100                # Dimenision des embeddings

**Gérer les fichiers**<br>
Le modèle seq2seq doit avoir 03 fichiers <br>
- input encodeur : phrase en anglais <br>
- input decodeur : phrase en français précédé du token \<sos\> <br>
- output decodeur : phrase en français suivi du token \<eos\> <br>

Exemple de phrase dans fra.txt : <br>
I want you to do something about it right away.	Je veux que tu y fasses quelque chose derechef.	CC-BY 2.0 (France)

In [ ]:
# Lire les fichiers
input_sentences = []
output_sentences = []
output_sentences_input = []
count = 0
with open('fra.txt') as f:
  for line in f.readlines():
    count+=1
    if count > NUM_SENTENCES:
      break
    if '\t' not in line:
      continue
    else:
      input_sentence, output, source= line.strip().split('\t')
      input_sentence = input_sentence
      output_sentence = output+'<eos>'
      output_sentence_input = '<sos>'+output
      input_sentences.append(input_sentence)
      output_sentences.append(output_sentence)
      output_sentences_input.append(output_sentence_input)

print("Input sentence 1: \n", input_sentences[0])
print("Output sentence 1: \n", output_sentences[0])
print("Output sentence input 1: \n", output_sentences_input[0])

Input sentence 1: 
 Go.
Output sentence 1: 
 Va !<eos>
Output sentence input 1: 
 <sos>Va !


**Représentation numérique des textes**

In [ ]:
# Numériser les inputs de l'encodeur (phrase en anglais) 

input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)                                # Tokenizer avec keras avec la taille du vocab
input_tokenizer.fit_on_texts(input_sentences)                                       # Entraîner le tokenizer sur les données d'entrée
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)             # Transformer le texte en format numérique
word2idx_inputs = input_tokenizer.word_index                                        # Recupérer l'ensemble des ids (chaque mot est représenté par un id) un dictionnaire keys=mots, values=ids
max_len_inputs = max(len(s) for s in input_sentences)                               # Taille max des sentences
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_len_inputs)   # Avoir des vecteurs de même taille (max_len_inputs)
num_words_input = len(word2idx_inputs) + 1                                          # Taille du vocabulaire +1 pour les embeddings
encoder_input_sequences                                                             

array([[ 0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0, 19],
       [ 0,  0,  0,  0,  0,  0, 19],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0, 28],
       [ 0,  0,  0,  0,  0,  0, 29],
       [ 0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0,  0, 30],
 

In [ ]:
# Numériser les inputs du décodeur (entrées et sorties) en français 

output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
output_tokenizer.fit_on_texts(output_sentences+output_sentences_input)                 # Entraîner le tokenizer sur la concaténation des deux listes
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)             # Numériser les sorties du décodeur
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_input) # Numériser les entrées du décodeur
word2idx_outputs = output_tokenizer.word_index                                         # Les ids des entrées et sorties du décodeur
max_len_outputs = max(len(s) for s in output_sentences)                                # Taille de la plus longue phrase
num_words_output = len(word2idx_outputs)+1                                             # Taiile du vocabulaire + 1
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_len_outputs, padding='post') # vecteurs de même taille (entrées)
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_len_outputs, padding='post')      # vecteurs de même taille (sorties)


In [ ]:
# Exemple de concaténation de liste
[1,2]+[3]

[1, 2, 3]

**One Hot Encoding pour les sorties du décodeur**<br>
Dans la section précédente, les sorties de l'encodeur avaient déjà une représentation numérique. Chaque phrase était représentée par un vecteur de taille **max_len_outputs** avec des ids qui remplacent les mots. Les ids sont compris entre 0 et la taille du vocabulaire.<br>

Cette représentation va être changée par le **One Hot Encoding** : <br>
Chaque phrase est maintenant représenté par une matrice de taille **max_len_outputs*num_words_output**. Chaque ligne de la matrice correspond à un mot de la phrase (dans l'ordre), sur la ligne de la matrice y'aura au plus un seul 1 qui se trouve à la position de l'id du mot donné dans la représentation de la section précedente, le reste est mis à 0.

In [ ]:
decoder_target_one_hot = np.zeros((len(output_sentences), max_len_outputs, num_words_output))

for index, sentence in enumerate(decoder_output_sequences):
  for t, word in enumerate(sentence):
    decoder_target_one_hot[index, t, word] = 1
decoder_target_one_hot

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [ ]:
num_words_output # Taille du vocabulaire du décodeur (phrase en anglais)

108

**Construire l'encodeur**<br>
L'embedding prend en entrée une matrice 2D. Pour chaque vecteur (une phrase), il represente chaque mot par un vecteur donc une sortie de taille nombre_de_mot_de_la_phrase*EMBEDDING_SIZE (LE NOMBRE DE MOT DES PHRASES EST max_len_inputs).<br>

sur **LSTM**, return_sequences = False, donc chaque phrase va être représentée en sortie par un vecteur de taille LSTM_NODES, et sur le batch, **encoder_outputs**, sera de taille (BATCH_SIZE, LSTM_NODES).<br>
Si return_sequences=True, la sortie pour chaque phrase sera une matrice 2D au lieu d'un vecteur max_len_inputs*LSTM_NODES et encoder_outputs de taille (BATCH_SIZE, MAX_LEN_INPUTS, LSTM_NODES).<br>

h correspond à l'état de la cellule, si return_sequences = False, il correspond à encoder_outputs, sinon il correspond à la dernière ligne de la matrice pour chaque sortie du LSTM sur une phrase. C'est un vecteur de taille LSTM_NODES, sur le batch (BATCH_SIZE, LSTM_NODES).<br>

c correspond à la mémoire à long terme, il est de même taille que h.<br>

**LSTM outputs on keras**
1. Default: Last Hidden State (Hidden State of the last time step)<br>
2. return_sequences=True : All Hidden States (Hidden State of ALL the time steps)<br>
3. return_state=True : Last Hidden State+ Last Hidden State (again!) + Last Cell State (Cell State of the last time step)<br>
4. return_sequences=True + return_state=True: All Hidden States (Hidden State of ALL the time steps) + Last Hidden State + Last Cell State (Cell State of the last time step)

In [ ]:
input_encoder = Input((max_len_inputs,))
encoder_embbeding = Embedding(input_dim=num_words_input, input_length=max_len_inputs,output_dim=EMBEDDING_SIZE)
encoder_input_x = encoder_embbeding(input_encoder)
encoder = LSTM(LSTM_NODES, return_state=True)
encoder_outputs,h,c = encoder(encoder_input_x)
encoder_states = [h,c] # h et c des vecteurs de taille LSTM_NODES

'''
si return_sequences = False : (batch_size, units)
si return_sequences = True : (batch_size, timestamps, units)
'''

**Construction du décodeur**<br>
Entrée embedding : (BATCH_SIZE, max_len_outputs)<br>
Sortie embedding : (BATCH_SIZE, max_len_outputs, LSTM_NODES)<br>

Entrée embedding : sortie embedding<br>
Sortie decoder_outputs : (BATCH_SIZE, max_len_outputs, LSTM_NODES)<br>

Entrée decoder_dense (MLP) : Sortie decoder_outputs<br>
Sortie decoder_dense : (BATCH_SIZE, max_len_outputs, num_words_output)

In [ ]:
input_decoder = Input((max_len_outputs,))
decoder_embbeding = Embedding(num_words_output, LSTM_NODES)
decoder_input_x = decoder_embbeding(input_decoder)
decoder_lstm = LSTM(LSTM_NODES,return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_states)
decoder_dense = Dense(num_words_output, activation='softmax')  # Couche pour la prédiction 
decoder_outputs = decoder_dense(decoder_outputs)

**Construction du modèle seq2seq**

In [ ]:
model = Model([input_encoder, input_decoder], decoder_outputs)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
[input_encoder, input_decoder] # [(BATCH_SIZE, 7), (BATCH_SIZE, 35)] 

[<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 35) dtype=float32 (created by layer 'input_4')>]

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 35)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 7, 100)       3800        input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 35, 256)      27648       input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() # Exemple d'un jeu de données avec keras

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
## train the model
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_target_one_hot,
    batch_size=10,
    epochs=EPOCHS,
    validation_split=0.1
)

Epoch 1/20
9/9 [==============================] - 11s 291ms/step - loss: 2.5623 - accuracy: 0.6531 - val_loss: 0.3810 - val_accuracy: 0.9200
Epoch 2/20
9/9 [==============================] - 1s 134ms/step - loss: 0.3523 - accuracy: 0.9282 - val_loss: 0.3722 - val_accuracy: 0.9371
Epoch 3/20
9/9 [==============================] - 1s 134ms/step - loss: 0.2902 - accuracy: 0.9424 - val_loss: 0.3705 - val_accuracy: 0.9429
Epoch 4/20
9/9 [==============================] - 1s 153ms/step - loss: 0.2617 - accuracy: 0.9489 - val_loss: 0.3820 - val_accuracy: 0.9429
Epoch 5/20
9/9 [==============================] - 1s 143ms/step - loss: 0.2614 - accuracy: 0.9465 - val_loss: 0.4045 - val_accuracy: 0.9429
Epoch 6/20
9/9 [==============================] - 1s 148ms/step - loss: 0.2385 - accuracy: 0.9543 - val_loss: 0.4019 - val_accuracy: 0.9429
Epoch 7/20
9/9 [==============================] - 1s 153ms/step - loss: 0.2544 - accuracy: 0.9514 - val_loss: 0.4249 - val_accuracy: 0.9429
Epoch 8/20
9/9 [===

Une fois l'entraînement du modèle seq2seq fini, les poids sont obtenus.<br>
On peut séparer l'encodeur et le décodeur maintenant.

**Construire le modèle de l'encodeur**

In [ ]:
# Pas besoin d'entraîner le modèle, car les poids sont déjà obtenus avec le modèle seq2seq
encoder_model = Model(input_encoder, encoder_states) # encoder_states = [h,c]
encoder_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 7, 100)            3800      
_________________________________________________________________
lstm_2 (LSTM)                [(None, 256), (None, 256) 365568    
Total params: 369,368
Trainable params: 369,368
Non-trainable params: 0
_________________________________________________________________


**Construire le modèle du décodeur**<br>

Le décodeur est initialisé par [h, c] de l'encodeur.<br>
La prédiction se fait mot par mot.<br>
decoder_embbeding(decoder_inputs_single) peut être utilisé avec un vecteur de talle 1 car dans l'initialisation de decoder_embedding, la taille des vecteurs d'entrée n'était pas spécifiée donc None.<br>

Sortie decoder_embedding : (BATCH_SIZE, 1, LSTM_NODES)<br>
decoder_outputs : (BATCH_SIZE, 1, LSTM_NODES)<br>
h : (BATCH_SIZE, LSTM_NODES)<br>
c : (BATCH_SIZE, LSTM_NODES)<br>
decoder_outputs : (BATCH_SIZE, 1, num_words_output)

In [ ]:
# La prédiction se fait mot par mot
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embbeding(decoder_inputs_single) # decoder_embbeding = Embedding(num_words_output, LSTM_NODES)

decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs) # decoder_lstm = LSTM(LSTM_NODES,return_state=True, return_sequences=True)

decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs) # decoder_dense = Dense(num_words_output, activation='softmax')

decoder_model = Model(
    [decoder_inputs_single]+decoder_states_inputs, # équivalent à [decoder_inputs_single, decoder_state_input_h, decoder_state_input_c]
    [decoder_outputs]+decoder_states
)
decoder_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         multiple             27648       input_13[0][0]                   
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 256)]        0                                            
____________________________________________________________________________________________

In [ ]:
[decoder_inputs_single]+decoder_states_inputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_13')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_11')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_12')>]

In [ ]:
[decoder_inputs_single,decoder_states_inputs]

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_13')>,
 [<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_11')>,
  <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_12')>]]

In [ ]:
[decoder_outputs]+decoder_states

[<KerasTensor: shape=(None, 1, 108) dtype=float32 (created by layer 'dense')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_3')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_3')>]

In [ ]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [ ]:
output_tokens = 0
def translate_sentence(input_seq):
    global output_tokens
    states_value = encoder_model.predict(input_seq)     # states_value = [h,c]
    target_seq = np.zeros((1, 1))                       # array([[0.]])
    target_seq[0, 0] = word2idx_outputs['sos']
    eos = word2idx_outputs['eos']
    output_sentence = []

    for _ in range(max_len_outputs):
        output_tokens, h, c = decoder_model.predict([target_seq]+states_value)
        idx = np.argmax(output_tokens[0, 0, :]) # proba (batch, 35,108)

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [ ]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1] # array([[values]])
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

-
Input: Go now.
Response: détends toi


In [ ]:
encoder_model.predict(input_seq) # (h,c)

[array([[ 7.91700780e-02, -3.59793752e-02, -4.47292656e-01,
         -2.20333815e-01,  3.16692531e-01,  5.41000187e-01,
         -3.93998176e-02,  1.63602665e-01, -3.88725728e-01,
         -3.48533422e-01,  6.02608882e-02, -7.21993372e-02,
         -1.79167032e-01,  3.52856487e-01, -9.08221304e-02,
          3.29001814e-01,  2.64196336e-01, -1.44088805e-01,
         -9.17000055e-01,  1.18501559e-01,  1.18383452e-01,
          5.10759912e-02,  1.94317680e-02,  3.46476912e-01,
          2.28997204e-03, -5.66138886e-02,  2.71178216e-01,
          1.80324361e-01,  5.28350510e-02,  6.31491467e-02,
          3.76654387e-01, -7.64216065e-01, -1.62652193e-03,
         -1.31664440e-01, -3.26158613e-01,  6.29956186e-01,
         -1.68976765e-02,  1.99221130e-02, -1.95001632e-01,
         -2.79583409e-02, -6.82948351e-01, -2.99464971e-01,
          1.67191550e-01, -4.28778291e-01,  7.64902174e-01,
         -4.88313548e-02, -3.95147860e-01, -1.87446494e-02,
          6.87573701e-02,  1.48658037e-0

In [ ]:
output_tokens

array([[[0.00917078, 0.00928607, 0.00883926, 0.00946218, 0.00953896,
         0.00953365, 0.00919233, 0.00931299, 0.00948264, 0.00942557,
         0.00944672, 0.00921851, 0.00929877, 0.00916996, 0.00928405,
         0.00944088, 0.00924674, 0.00936707, 0.00939569, 0.00934956,
         0.00916197, 0.00934398, 0.00937026, 0.00933487, 0.00927241,
         0.00929015, 0.00932668, 0.00932862, 0.00932082, 0.00920961,
         0.0094407 , 0.00927543, 0.00955082, 0.00937454, 0.00926919,
         0.00962851, 0.00939844, 0.00919239, 0.00927307, 0.00928615,
         0.00935162, 0.00950285, 0.00930571, 0.0090495 , 0.00921396,
         0.00940089, 0.00928133, 0.009334  , 0.0093665 , 0.00927674,
         0.00917016, 0.00925401, 0.00921759, 0.00935428, 0.00940239,
         0.00928428, 0.0093721 , 0.0091899 , 0.00911624, 0.00925829,
         0.00923131, 0.00947088, 0.00930714, 0.00946948, 0.00928259,
         0.00916231, 0.00941814, 0.00936416, 0.00934095, 0.00933816,
         0.00924574, 0.00924785, 0

In [ ]:
Comment créer un modèle seq2se?
"""
1) Ajouter les balises <sos> et <eos> sur chaque début et fin de phrase du target.

2) Créer un modèle générique qui fusionne l'encoder et le décoder donc compter 2 Input:
   - L'encoder a un Input avec la taille des phrases de la source, une couche d'embedding, une couche de LSTM
     avec return_state = True pour capter l'état de l'encoder qu'on va envoyer au decoder (h,c).
   - Le decoder a un Input avec la taille des phrases du target précédées de <sos>, une couche d'embedding,
     couche de LSTM avec initial_state = (h,c), (h,c) est founi par la couche LSTM de l'encoder, return_sequences = True
     pour retourner (batch_size, timestamp, units); et une couche dense qui retourne une phrase prédite.
     
3) Entrainer le modèle ainsi créé.     

4) Créer un modèle spécifique pour l'encoder contenant une couche Input prenant une sentence de la source, 
   une couche d'Embedding et une couche de LSTM avec return_state = True pour garder (h,c).
   
5) Créer un modèle spécifique pour le decoder contenant une couche Input prenant [un mot du target + 
   (h=Input,c=Input)], une couche d'Embedding, une couche de LSTM avec initial_state = (h,c) et 
   return_sequences = True, une couche Dense qui est connectée avec la couche LSTM (batch_size, timestamps,units)
   
6) Pour faire fonctionner le modèle, on donne une phrase du source à la fonction qui fait des boucles 
   sur le décoder afin de prédire mot par mot et pas sentence par sentence.
"""

(1, 1, 108)

In [ ]:
num_words_output

108

In [ ]:
np.argmax(output_tokens)

35

In [1]:
import numpy as np

In [4]:
np.array([1,2,3]).shape

(3,)

In [2]:
 np.array([np.zeros((1, 1))]+[np.array([1,2,3]), np.array([4,5,5])]).shape

/home/faye/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(3,)

**NB : les inputs doivent minimum être en 2D**